# Power Plant Data Product: Extraction Step 1

This notebook covers the first step of the extraction pipeline:
* Load external source CSV source from WRI Global Power Plant Database
* Connect to pachyderm service, create a new repository if need be and check in source data
* Verify that the file can be accessed for the next step which is data loading

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
# Report exceptions only in cells output
%xmode Minimal

Exception reporting mode: Minimal


In [2]:
# For loading predefined environment variables from files
# including sensitive access credentials in Development environment
from dotenv import load_dotenv

import os
import pathlib
import python_pachyderm

Define Environment and Execution Variables

In [3]:
# Load environment variables from credentials.env
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

Read the data directory holding raw source data, and list all files

In [4]:
path = '../data/global_power_plant_database_v_1_3/'
files = os.listdir(path)
for f in files:
    print(f)

A_Global_Database_of_Power_Plants.pdf
Estimating_Power_Plant_Generation_in_the_Global_Power_Plant_Database.pdf
README.txt
RELEASE_NOTES.txt
global_power_plant_database.csv


Connect to Pachyderm service using the Python client, create a new repo and check in the source files

In [5]:
# Create a pachyderm client by manually setting up host and port
client = python_pachyderm.Client(os.environ['PACH_ENDPOINT'], os.environ['PACH_PORT'])

In [6]:
# Create a new pachyderm repo called `gppd`
client.create_repo("wri-gppd")

In [7]:
# Display list of repos
print(list(client.list_repo()))

[repo {
  name: "wri-gppd"
  type: "user"
  project {
  }
}
created {
  seconds: 1687256504
  nanos: 671215000
}
]


In [8]:
with client.commit("wri-gppd", "master") as commit:
        # Add all the files, recursively inserting from the directory
        # Alternatively, we could use `client.put_file_url` or
        # `client_put_file_bytes`.
        python_pachyderm.put_files(client, path, commit, "/global_power_plant_database_v_1_3/")

In [9]:
# Check the file commit
print(list(client.list_file(("wri-gppd","master"), "/global_power_plant_database_v_1_3/")))

[file {
  commit {
    branch {
      repo {
        name: "wri-gppd"
        type: "user"
        project {
        }
      }
      name: "master"
    }
    id: "2551a4e5227b418bb6f0f75f74577372"
  }
  path: "/global_power_plant_database_v_1_3/A_Global_Database_of_Power_Plants.pdf"
  datum: "default"
}
file_type: FILE
committed {
  seconds: 1687256511
  nanos: 860898000
}
size_bytes: 1441800
hash: ".M\370\311\026`\334\001\000?\342>B\0351\373\021\322\276\205\025\231\200\262\270\"H\330|\260\314\037"
, file {
  commit {
    branch {
      repo {
        name: "wri-gppd"
        type: "user"
        project {
        }
      }
      name: "master"
    }
    id: "2551a4e5227b418bb6f0f75f74577372"
  }
  path: "/global_power_plant_database_v_1_3/Estimating_Power_Plant_Generation_in_the_Global_Power_Plant_Database.pdf"
  datum: "default"
}
file_type: FILE
committed {
  seconds: 1687256511
  nanos: 860898000
}
size_bytes: 1685244
hash: "\331\241\363O-\021\232\362r\375P\313\235\356\363\223\342\

In [10]:
# Delete pachyderm repo called `wri-gppd`
# Only required if you need to run the pipeline again
# client.delete_repo("wri-gppd")